# Preparing the data

## Importing libraries, downloading the model

In [ ]:
import pandas
import sklearn
import numpy

print(pandas.__version__)
print(sklearn.__version__)
print(numpy.__version__)

1.5.3
1.2.2
1.22.4


In [ ]:
import spacy
# Small Russian model:
# !python -m spacy download ru_core_news_sm
# nlp = spacy.load('ru_core_news_sm')

# Large Russian model:
!python -m spacy download ru_core_news_lg
nlp = spacy.load('ru_core_news_lg')

2023-06-15 18:48:19.646627: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.4/513.4 MB 2.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')


## Making lists and doc objects from csv files

In [ ]:
# Let's load the training data from a csv file
train_set = pandas.read_csv('./train_data.csv', encoding='utf-8')
# train_set

In [ ]:
test_set = pandas.read_csv('./test_data.csv', encoding='utf-8')
# test_set

In [ ]:
train_sentences = train_set['text'].to_list()
train_authors = train_set['author'].to_list()

test_sentences = test_set['text'].to_list()
test_authors = test_set['author'].to_list()

print(len(train_authors), len(test_authors))

10000 1000


In [ ]:
train_doc_sentences = nlp.pipe(train_sentences)
test_doc_sentences = nlp.pipe(test_sentences)

## Preparing the feature set

In [ ]:
def fivegram_pos_extractor_unique(list_of_doc_sentences):
    n = 5
    fivegram_pos_tags = []
    for doc in list_of_doc_sentences:
        # Iterate over each possible fivegram in the document
        for i in range(len(doc) - n + 1):
            # Extract the tokens for the current fivegram
            fivegram_tokens = doc[i : i + n]
            # Extract the POS tags of the tokens and add the POS tag combination to the list
            fivegram_pos = tuple(token.pos_ for token in fivegram_tokens)
            fivegram_pos_tags.append(fivegram_pos)
    unique_fivegram_pos_tags = list(set(fivegram_pos_tags))

    return unique_fivegram_pos_tags

def fivegram_pos_extractor_from_sentence(doc):
    n = 5
    fivegram_pos_tags = []
    for i in range(len(doc) - n + 1):
    # Extract the tokens for the current fivegram
        fivegram_tokens = doc[i : i + n]
        # Extract the POS tags of the tokens and add the POS tag combination to the list
        fivegram_pos = tuple(token.pos_ for token in fivegram_tokens)
        fivegram_pos_tags.append(fivegram_pos)
    unique_fivegram_pos_tags = list(set(fivegram_pos_tags))

    return unique_fivegram_pos_tags

In [ ]:
train_doc_sentences = nlp.pipe(train_sentences)

unique_5gram_POS_seq = fivegram_pos_extractor_unique(train_doc_sentences)
print(unique_5gram_POS_seq[:10])
print(len(unique_5gram_POS_seq))

[('VERB', 'PUNCT', 'SPACE', 'ADJ', 'ADP'), ('VERB', 'PART', 'NOUN', 'PROPN', 'PUNCT'), ('NOUN', 'PROPN', 'PROPN', 'CCONJ', 'ADP'), ('PUNCT', 'PUNCT', 'PART', 'DET', 'ADJ'), ('PUNCT', 'PART', 'PRON', 'PRON', 'ADJ'), ('ADP', 'PRON', 'NOUN', 'VERB', 'PRON'), ('PART', 'PUNCT', 'SPACE', 'NOUN', 'PRON'), ('INTJ', 'NOUN', 'PUNCT', 'PART', 'PROPN'), ('ADP', 'PRON', 'NOUN', 'PRON', 'PRON'), ('NOUN', 'PART', 'VERB', 'PRON', 'ADJ')]
49912


In [ ]:
import numpy

# We are creating a matrix with zero vectors for each review (in training set and test set)
train_features_matrix = numpy.zeros((len(train_sentences), len(unique_5gram_POS_seq)))
print(train_features_matrix.shape)

test_features_matrix = numpy.zeros((len(test_sentences), len(unique_5gram_POS_seq)))
print(test_features_matrix.shape)

(10000, 49912)
(1000, 49912)


# Modifying the feature vectors

## Visualisation tests

In [ ]:
import sys

train_doc_sentences = nlp.pipe(train_sentences)

for sentence, author in zip(train_doc_sentences, train_authors):
    print(sentence)
    fivegram_pos_list = fivegram_pos_extractor_from_sentence(sentence)
    print(fivegram_pos_list)
    for fivegram in unique_5gram_POS_seq:
      if fivegram in fivegram_pos_list:
        print(fivegram)
        fivegram_id = unique_5gram_POS_seq.index(fivegram)
        print(fivegram_id)
        sys.exit()

Но каково же было мое изумление, когда Наташа с первых же слов остановила меня и сказала, что нечего ее утешать, что она уже пять дней, как знает про это..     – Боже мой!
[('NOUN', 'PUNCT', 'SCONJ', 'PROPN', 'ADP'), ('SCONJ', 'VERB', 'PRON', 'VERB', 'PUNCT'), ('DET', 'NOUN', 'PUNCT', 'SCONJ', 'PROPN'), ('VERB', 'PUNCT', 'SCONJ', 'VERB', 'PRON'), ('PUNCT', 'SPACE', 'PUNCT', 'NOUN', 'DET'), ('SCONJ', 'PROPN', 'ADP', 'ADJ', 'PART'), ('ADV', 'NUM', 'NOUN', 'PUNCT', 'SCONJ'), ('CCONJ', 'ADJ', 'PART', 'AUX', 'DET'), ('PART', 'AUX', 'DET', 'NOUN', 'PUNCT'), ('CCONJ', 'VERB', 'PUNCT', 'SCONJ', 'VERB'), ('PROPN', 'ADP', 'ADJ', 'PART', 'NOUN'), ('PUNCT', 'SCONJ', 'PROPN', 'ADP', 'ADJ'), ('SCONJ', 'VERB', 'ADP', 'PRON', 'PUNCT'), ('NOUN', 'VERB', 'PRON', 'CCONJ', 'VERB'), ('PRON', 'PUNCT', 'SPACE', 'PUNCT', 'NOUN'), ('PART', 'NOUN', 'VERB', 'PRON', 'CCONJ'), ('PRON', 'VERB', 'PUNCT', 'SCONJ', 'PRON'), ('VERB', 'PUNCT', 'SCONJ', 'PRON', 'ADV'), ('NUM', 'NOUN', 'PUNCT', 'SCONJ', 'VERB'), ('PUNCT',

SystemExit: ignored

Visualisation test 2:

In [ ]:
import sys

train_doc_sentences = nlp.pipe(train_sentences)

for sentence, author, feature_vector in zip(train_doc_sentences, train_authors, train_features_matrix):
    print('Author:', author)
    print(sentence)
    fivegram_pos_list = fivegram_pos_extractor_from_sentence(sentence)
    print(fivegram_pos_list)
    for fivegram in unique_5gram_POS_seq:
      if fivegram in fivegram_pos_list:
        fivegram_id = unique_5gram_POS_seq.index(fivegram)
        print(fivegram_id)
        print(fivegram)
        feature_vector[fivegram_id] = 1
        print(feature_vector.tolist())
        sys.exit()


Author: Dostoevsky
Но каково же было мое изумление, когда Наташа с первых же слов остановила меня и сказала, что нечего ее утешать, что она уже пять дней, как знает про это..     – Боже мой!
[('NOUN', 'PUNCT', 'SCONJ', 'PROPN', 'ADP'), ('SCONJ', 'VERB', 'PRON', 'VERB', 'PUNCT'), ('DET', 'NOUN', 'PUNCT', 'SCONJ', 'PROPN'), ('VERB', 'PUNCT', 'SCONJ', 'VERB', 'PRON'), ('PUNCT', 'SPACE', 'PUNCT', 'NOUN', 'DET'), ('SCONJ', 'PROPN', 'ADP', 'ADJ', 'PART'), ('ADV', 'NUM', 'NOUN', 'PUNCT', 'SCONJ'), ('CCONJ', 'ADJ', 'PART', 'AUX', 'DET'), ('PART', 'AUX', 'DET', 'NOUN', 'PUNCT'), ('CCONJ', 'VERB', 'PUNCT', 'SCONJ', 'VERB'), ('PROPN', 'ADP', 'ADJ', 'PART', 'NOUN'), ('PUNCT', 'SCONJ', 'PROPN', 'ADP', 'ADJ'), ('SCONJ', 'VERB', 'ADP', 'PRON', 'PUNCT'), ('NOUN', 'VERB', 'PRON', 'CCONJ', 'VERB'), ('PRON', 'PUNCT', 'SPACE', 'PUNCT', 'NOUN'), ('PART', 'NOUN', 'VERB', 'PRON', 'CCONJ'), ('PRON', 'VERB', 'PUNCT', 'SCONJ', 'PRON'), ('VERB', 'PUNCT', 'SCONJ', 'PRON', 'ADV'), ('NUM', 'NOUN', 'PUNCT', 'SCONJ',

SystemExit: ignored

## Writing a function for feature vector modification

In [ ]:
def modify_feature_vectors(doc_sentences, features_matrix):
  for sentence, feature_vector in zip(doc_sentences, features_matrix):
    fivegram_pos_list = fivegram_pos_extractor_from_sentence(sentence)
    for fivegram in unique_5gram_POS_seq:
      if fivegram in fivegram_pos_list:
        fivegram_id = unique_5gram_POS_seq.index(fivegram)
        feature_vector[fivegram_id] = 1
  return features_matrix

In [ ]:
train_features_matrix = numpy.zeros((len(train_sentences), len(unique_5gram_POS_seq)))
train_doc_sentences = nlp.pipe(train_sentences)

In [ ]:
train_features_matrix_final = modify_feature_vectors(train_doc_sentences, train_features_matrix)

# Training

In [ ]:
from sklearn.linear_model import LogisticRegression

lr_5gram_pos = LogisticRegression()

# Train the model on the data, storing the information learned from the dat`a
# Model is learning the relationship between digits (x_train) and labels (y_train)
lr_5gram_pos.fit(train_features_matrix_final, train_authors)

print(lr_5gram_pos.classes_)
print(lr_5gram_pos.get_params())

['Chekhov' 'Dostoevsky' 'Gogol' 'Tolstoy']
{'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Modifying the test set feature vectors

In [ ]:
test_doc_sentences = nlp.pipe(test_sentences)
test_features_matrix = numpy.zeros((len(test_sentences), len(unique_5gram_POS_seq)))

test_features_matrix_final = modify_feature_vectors(test_doc_sentences, test_features_matrix)

# Making predictions

In [ ]:
def predict(i):
    print(test_sentences[i])
    # print the features of the index
    print(test_features_matrix_final[i])
    # print the correct label of the index
    print(test_authors[i])

    print()
    print("Prediction:")
    # print the prediction for the features of this index
    print(lr_5gram_pos.predict([test_features_matrix_final[i]]))
    # print the probabilities for each label predictions
    print(lr_5gram_pos.predict_proba([test_features_matrix_final[i]]))
    print()

In [ ]:
predict(0)
predict(1)
predict(2)
predict(3)
predict(4)
predict(5)

In [ ]:
test_predictions_pos = lr_5gram_pos.predict(test_features_matrix_final)

for p, r in zip(test_predictions_pos[:10], test_authors[:10]):
    if p == r:
        result = "Correct"
    else:
        result = "Incorrect"
    print(p + "(" + result + ":" + r + ")")

Gogol(Incorrect:Dostoevsky)
Gogol(Incorrect:Chekhov)
Gogol(Correct:Gogol)
Chekhov(Incorrect:Dostoevsky)
Gogol(Correct:Gogol)
Dostoevsky(Correct:Dostoevsky)
Dostoevsky(Correct:Dostoevsky)
Chekhov(Correct:Chekhov)
Tolstoy(Incorrect:Dostoevsky)
Chekhov(Incorrect:Dostoevsky)


# Saving the model

In [ ]:
import pickle

# Save to file in the current working directory
pkl_filename = "logreg_POS.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(lr_5gram_pos, file)

# Evaluating the model

## Dummy model

In [ ]:
from sklearn.metrics import accuracy_score

dummy_predictions = ['Dostoevsky'] * len(test_sentences)
print(len(dummy_predictions))

# Calculate the accuracy of these "dummy predictions"
acc_dummy = accuracy_score(test_authors, dummy_predictions)
print('The accuracy is:', acc_dummy)
print()

from sklearn.metrics import classification_report
print(classification_report(test_authors, dummy_predictions))

1000
The accuracy is: 0.25

              precision    recall  f1-score   support

     Chekhov       0.00      0.00      0.00       250
  Dostoevsky       0.25      1.00      0.40       250
       Gogol       0.00      0.00      0.00       250
     Tolstoy       0.00      0.00      0.00       250

    accuracy                           0.25      1000
   macro avg       0.06      0.25      0.10      1000
weighted avg       0.06      0.25      0.10      1000



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## POS Model

In [ ]:
from sklearn.metrics import accuracy_score

print('Accuracy:')

acc = accuracy_score(test_authors, test_predictions_pos)
print(acc)
corr_count = accuracy_score(test_authors, test_predictions_pos, normalize=False)
total_count = len(test_authors)

print("Total reviews: " + str(str(total_count)))
print("Total correct predictions:" + str(corr_count))
corr_ratio = corr_count / total_count
print("Correct ratio:" + str(corr_ratio))

Accuracy:
0.365
Total reviews: 1000
Total correct predictions:365
Correct ratio:0.365


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_authors, test_predictions_pos))

              precision    recall  f1-score   support

     Chekhov       0.32      0.47      0.38       250
  Dostoevsky       0.43      0.36      0.39       250
       Gogol       0.35      0.32      0.33       250
     Tolstoy       0.40      0.32      0.35       250

    accuracy                           0.36      1000
   macro avg       0.37      0.37      0.36      1000
weighted avg       0.37      0.36      0.36      1000

